# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-04 00:00:00  to  2020-04-05 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,109,112,3,2.752294
1,02,223,221,-2,-0.896861
2,03,44,45,1,2.272727
3,04,29,31,2,6.896552
4,05,60,59,-1,-1.666667


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,2999,3075,76,2.534178
92,92,1929,1950,21,1.088647
94,94,1651,1729,78,4.724409
93,93,1310,1338,28,2.137405
69,69,1215,1250,35,2.880658


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -10  Mean:  7.71875  Max:  78


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -11.76470588235294  Mean %:  4.184063431411407  Max %:  34.84848484848485


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
1,02,223,221,-2,-0.896861
4,05,60,59,-1,-1.666667
10,11,102,95,-7,-6.862745
11,12,58,56,-2,-3.448276
18,19,43,42,-1,-2.325581
24,26,254,251,-3,-1.181102
25,27,69,68,-1,-1.449275
28,2A,80,78,-2,-2.500000
31,31,227,226,-1,-0.440529
37,37,170,166,-4,-2.352941


#### <br> Departments with highest evolution of hospitalizations (total number)

In [11]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
94,94,1651,1729,78,4.724409
75,75,2999,3075,76,2.534178
59,59,773,828,55,7.115136
12,13,1040,1095,55,5.288462
77,77,673,709,36,5.349183


#### <br> Departments with highest rate of hospitalizations (%)

In [12]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
30,30,66,89,23,34.848485
58,58,27,34,7,25.925926
86,86,45,56,11,24.444444
65,65,43,52,9,20.930233
8,09,15,18,3,20.000000


#### <br> Focus on some departments

In [13]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,2999,3075,76,2.534178


In [14]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,162,173,11,6.790123


In [15]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,227,226,-1,-0.440529
